In [2]:
import pandas as pd
import os
import imblearn

In [4]:
df = pd.read_csv("indian_liver_patient.csv", encoding = "unicode_escape")

In [5]:
df.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [6]:
df.shape

(583, 11)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


In [8]:
df.isna().sum()

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    4
Dataset                       0
dtype: int64

In [11]:
df["Gender"] .unique()

array([0, 1], dtype=int64)

In [12]:
df["Gender"] = df["Gender"].replace("Female",0) # 0 -female
df["Gender"] = df["Gender"].replace("Male",1) # 1 - male

In [13]:
# 0 -not suffering 1 - suffering with liver diesease
df["Result"] = df["Dataset"].replace(2,0)
df.drop("Dataset", axis = 1, inplace = True)

In [14]:
# filling missing values
df['Albumin_and_Globulin_Ratio'] = df['Albumin_and_Globulin_Ratio'].fillna(df['Albumin_and_Globulin_Ratio'].mean())

In [15]:
df["Result"].value_counts()

Result
1    416
0    167
Name: count, dtype: int64

In [16]:
X = df.drop('Result', axis=1)
y = df["Result"]

In [17]:
from imblearn.over_sampling import RandomOverSampler

In [18]:
# Over sampling due to imbalanced dataset
os = RandomOverSampler(random_state=42)

In [19]:
X_res,y_res = os.fit_resample(X,y)

In [20]:
X_res.shape, y_res.shape

((832, 10), (832,))

In [21]:
y_res.value_counts()

Result
1    416
0    416
Name: count, dtype: int64

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
rfc = RandomForestClassifier(n_estimators = 200, bootstrap = True)

In [26]:
model = rfc.fit(X_train,y_train)

In [27]:
y_pred = model.predict(X_test)

In [28]:
from sklearn.metrics import classification_report, confusion_matrix

In [31]:
print(confusion_matrix(y_test,y_pred))

[[82 10]
 [13 62]]


In [32]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.89      0.88        92
           1       0.86      0.83      0.84        75

    accuracy                           0.86       167
   macro avg       0.86      0.86      0.86       167
weighted avg       0.86      0.86      0.86       167

